In [1]:
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from continue_dataset import ShouldContinueDataset
import math
import models
from models import *
import os
import os.path
import numpy as np
import utils

In [2]:
"""
Train the network that predicts whether we should continue adding objects
"""
parser = argparse.ArgumentParser(description='Should Continue')
parser.add_argument('--data-dir', type=str, default="bedroom", metavar='S')
parser.add_argument('--num-workers', type=int, default=6, metavar='N')
parser.add_argument('--last-epoch', type=int, default=-1, metavar='N')
parser.add_argument('--train-size', type=int, default=6400, metavar='N')
parser.add_argument('--save-dir', type=str, default="train/bedroom", metavar='S')
parser.add_argument('--ablation', type=str, default=None, metavar='S')
parser.add_argument('--lr', type=float, default=0.001, metavar='N')
parser.add_argument('--eps', type=float, default=1e-6, metavar='N')
parser.add_argument('--use-count', action='store_true', default=False)

_StoreTrueAction(option_strings=['--use-count'], dest='use_count', nargs=0, const=True, default=False, type=None, choices=None, help=None, metavar=None)

In [3]:
args = parser.parse_args("")

In [4]:
save_dir = args.save_dir
utils.ensuredir(save_dir)
learning_rate = args.lr
batch_size = 16

In [11]:
suncg_root = "/home/ubuntu/research/suncg/"

In [12]:
with open(f"{suncg_root}/{args.data_dir}/final_categories_frequency", "r") as f:
    lines = f.readlines()
num_categories = len(lines)-2

In [13]:
num_categories

29

In [15]:
if args.ablation is None:
    num_input_channels = num_categories+8
elif args.ablation == "basic":
    num_input_channels = 6
elif args.ablation == "depth":
    num_input_channels = 1
else:
    raise NotImplementedError